In [1]:
%cd ..

/home/jbananafish/Desktop/Master/Thesis/code/gcnboost


In [2]:
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, Linear, GraphConv, GATConv, BatchNorm, to_hetero
import torch_geometric.transforms as T
import torch_geometric.nn as operators

from src.data.artgraph import ArtGraph

In [3]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [4]:
class HeteroGNN(torch.nn.Module):
    def __init__(self, operator=SAGEConv, activation=torch.nn.ReLU, hidden_channels=16, out_channels=300, num_layers=1, dropout=0.5, bn=False, skip=False):
        super(HeteroGNN, self).__init__()
        self.dropout = dropout
        self.bn = bn
        self.skip = skip
        self.activation = activation
        self.convs = torch.nn.ModuleList()
        self.lins = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = operator((-1, -1), hidden_channels)
            lin = Linear(-1, hidden_channels)
            bn = torch.nn.BatchNorm1d(hidden_channels)
            self.convs.append(conv)
            self.lins.append(lin)
            self.bns.append(bn)
        self.conv_out = operator((-1, -1), out_channels)

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            if self.skip:
                x = conv(x, edge_index) + self.lins[i](x)
            else:
                x = conv(x, edge_index)
                
            if self.bn:
                x = self.bns[i](x)
            x = self.activation(x)

            if self.training:
                x = F.dropout(x, self.dropout)
        x = self.conv_out(x, edge_index)
        return F.log_softmax(x, dim=1)

In [5]:
class HeteroSGNN(torch.nn.Module):
    def __init__(self, operator, activation, aggr, hidden_channels, out_channels, metadata, n_layers, dropout, bn, skip):
        super(HeteroSGNN, self).__init__()
        self.gnn = HeteroGNN(operator, activation, hidden_channels, out_channels, n_layers, dropout, bn, skip)
        self.gnn = to_hetero(self.gnn, metadata, aggr=aggr)

    def forward(self, x, edge_index):
        return [self.gnn(x, edge_index)]

class HeteroMGNN(torch.nn.Module):
    def __init__(self, operator, activation, aggr, hidden_channels, out_channels, metadata, n_layers, dropout, bn, skip):
        super(HeteroMGNN, self).__init__()
        self.gnn_artist = HeteroGNN(operator, activation, hidden_channels, out_channels['artist'], n_layers, dropout, bn, skip)
        self.gnn_artist = to_hetero(self.gnn_artist, metadata, aggr=aggr)

        self.gnn_style = HeteroGNN(operator, activation, hidden_channels, out_channels['style'], n_layers, dropout, bn, skip)
        self.gnn_style = to_hetero(self.gnn_style, metadata, aggr=aggr)

        self.gnn_genre = HeteroGNN(operator, activation, hidden_channels, out_channels['genre'], n_layers, dropout, bn, skip)
        self.gnn_genre = to_hetero(self.gnn_genre, metadata,aggr=aggr)

    def forward(self, x, edge_index):
        return [self.gnn_artist(x, edge_index), self.gnn_style(x, edge_index), self.gnn_genre(x, edge_index)]

In [18]:
class ArtGraphGCNBoost:

    operator_registry = {
        'SAGEConv': operators.SAGEConv,
        'GraphConv': operators.GraphConv,
        'GATConv': operators.GATConv,
        'GCNConv': operators.GCNConv
    }
    
    activation_registry = {
        'relu': torch.nn.ReLU(),
        'prelu': torch.nn.PReLU()
    }

    map_id2labels = {
        0: 'artist',
        1: 'style',
        2: 'genre'
    }

    map_labels2id = {
        'artist': 0,
        'style': 1,
        'genre': 2
    }

    def __init__(self, args, graph_type='hetero', training_mode='multi_task'):
        
        self.graph_type = graph_type
        self.traning_mode = training_mode
        assert graph_type in ['hetero', 'homo']
        assert training_mode in ['multi_task', 'single_task']
        assert args.operator in self.operator_registry.keys()

        self.base_data, self.data, self.y, self.model, self.optimizer = self._bootstrap(args)
        self.artworks = self.base_data[0]['artwork']
        self.train_mask = self.artworks.train_mask
        self.val_mask = self.artworks.val_mask
        self.test_mask = self.artworks.test_mask

    def _bootstrap(self, args):
        base_data = ArtGraph("data", preprocess='constant', features=True, type='ekg')
        data = base_data[0]
        def del_some_nodes():
            del data['gallery']
            del data['city']
            del data['country']
            del data['auction']
            del data['media']
            del data['tag']
            del data['period']
            del data['serie']
            del data['subject']
            del data['people']
            del data['training_node']

            del data['artwork', 'auction_rel', 'auction']
            del data['city', 'country_rel', 'country']
            del data['gallery', 'city_rel', 'city']
            del data['gallery', 'country_rel', 'country']
            del data['artwork', 'locatedin_rel', 'gallery']
            del data['artwork', 'completedin_rel', 'city']

            del data['artwork', 'media_rel', 'media']
            del data['artwork', 'about_rel', 'tag']
            del data['artwork', 'period_rel', 'period']
            del data['artwork', 'serie_rel', 'serie']
            del data['people', 'patrons_rel', 'artist']

            del data['artist', 'influenced_rel', 'artist']
            del data['artist', 'subject_rel', 'subject']
            del data['artist', 'training_rel', 'artist']
            del data['artist', 'training_rel', 'training_node']
            del data['artist', 'patrons_rel', 'people']
            del data['artist', 'teacher_rel', 'artist']
        data = T.ToUndirected()(data)
        if self.traning_mode == 'multi_task':
            model = HeteroMGNN(operator=self.operator_registry[args.operator], 
                            activation=self.activation_registry[args.activation],
                            aggr=args.aggr, 
                            hidden_channels=args.hidden, 
                            out_channels=base_data.num_classes, 
                            metadata=data.metadata(),
                            n_layers=args.nlayers, 
                            dropout=args.dropout,
                            bn=args.bn,
                            skip=args.skip)
            y = torch.stack([base_data[0]['artwork'].y_artist, base_data[0]['artwork'].y_style, base_data[0]['artwork'].y_genre])
        if self.traning_mode == 'single_task':
            model = HeteroSGNN(operator=self.operator_registry[args.operator], 
                            activation=self.activation_registry[args.activation],
                            aggr=args.aggr, 
                            hidden_channels=args.hidden, 
                            out_channels=base_data.num_classes[args.label], 
                            metadata=data.metadata(),
                            n_layers=args.nlayers, 
                            dropout=args.dropout,
                            bn=args.bn,
                            skip=args.skip)
            y = torch.stack([base_data[0]['artwork'][f'y_{args.label}']])

        
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=3e-4)
        
        return base_data, data, y, model, optimizer

    def get_accuracy(self, predicted, labels):
        return predicted.argmax(dim=1).eq(labels).sum()/predicted.shape[0]

    def get_accuracies(self, predicted, labels, mask):
        accuracies = [] 
        for i, _ in enumerate(labels):
            accuracies.append(self.get_accuracy(predicted[i]['artwork'][mask], labels[i][mask]))
        return accuracies

    def get_loss(self, predicted, labels):
        return F.nll_loss(predicted, labels.type(torch.LongTensor))

    def get_losses(self, predicted, labels, mask):
        losses = []
        for i, _ in enumerate(labels):
            losses.append(self.get_loss(predicted[i]['artwork'][mask], labels[i][mask]))
        return losses

    def hetero_training(self):
        self.model.train()
        self.optimizer.zero_grad()
        out = self.model(self.data.x_dict, self.data.edge_index_dict)

        train_losses = self.get_losses(out, self.y, self.train_mask)
        train_total_loss = sum(train_losses)

        train_total_loss.backward()
        self.optimizer.step()

        train_accuracies = self.get_accuracies(out, self.y, self.train_mask)

        return out, train_losses, train_accuracies
        
    @torch.no_grad()
    def hetero_test(self):
        self.model.eval()
        out = self.model(self.data.x_dict, self.data.edge_index_dict)
        val_losses = self.get_losses(out, self.y, self.val_mask)
        test_losses = self.get_losses(out, self.y, self.test_mask)

        val_accuracies = self.get_accuracies(out, self.y, self.val_mask)
        test_accuracies = self.get_accuracies(out, self.y, self.test_mask)

        return val_losses, val_accuracies, test_losses, test_accuracies

In [20]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--exp', type=str, default='default', help='Experiment name.')
parser.add_argument('--type', type=str, default='hetero', help='Graph type (hetero|homo).')
parser.add_argument('--mode', type=str, default='single_task', help='Training mode (multi_task|single_task).')
parser.add_argument('--label', type=str, default='genre', help='Label to predict (artist|style|genre).')
parser.add_argument('--epochs', type=int, default=200, help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.')
parser.add_argument('--hidden', type=int, default=16, help='Number of hidden units.')
parser.add_argument('--nlayers', type=int, default=1, help='Number of layers.')
parser.add_argument('--dropout', type=float, default=0, help='Dropout rate (1 - keep probability).')
parser.add_argument('--operator', type=str, default='SAGEConv', help='The graph convolutional operator.')
parser.add_argument('--activation', type=str, default='relu', help='The activation function.')
parser.add_argument('--aggr', type=str, default='sum', help='Aggregation function.')
parser.add_argument('--skip', action='store_true', default='False', help='Add skip connection.')
parser.add_argument('--bn', action='store_true', default='False', help='Add batch normalization.')
args, unknown = parser.parse_known_args()

In [21]:
gcn = ArtGraphGCNBoost(args, training_mode=args.mode)

In [22]:
gcn.data

HeteroData(
  artwork={
    x=[61477, 128],
    y_artist=[61477],
    y_style=[61477],
    y_genre=[61477],
    train_mask=[61477],
    val_mask=[61477],
    test_mask=[61477]
  },
  artist={ x=[300, 128] },
  style={ x=[83, 128] },
  genre={ x=[50, 128] },
  field={ x=[65, 128] },
  movement={ x=[121, 128] },
  (artist, field_rel, field)={ edge_index=[2, 323] },
  (artist, movement_rel, movement)={ edge_index=[2, 286] },
  (artwork, genre_rel, genre)={ edge_index=[2, 61820] },
  (artwork, style_rel, style)={ edge_index=[2, 63157] },
  (artwork, author_rel, artist)={ edge_index=[2, 61477] },
  (field, rev_field_rel, artist)={ edge_index=[2, 323] },
  (movement, rev_movement_rel, artist)={ edge_index=[2, 286] },
  (genre, rev_genre_rel, artwork)={ edge_index=[2, 61820] },
  (style, rev_style_rel, artwork)={ edge_index=[2, 63157] },
  (artist, rev_author_rel, artwork)={ edge_index=[2, 61477] }
)

In [23]:
for epoch in tqdm(range(0, args.epochs)):
    out, train_losses, train_accuracies = gcn.hetero_training()
    val_losses, val_accuracies, test_losses, test_accuracies = gcn.hetero_test()
    if args.mode == 'multi_task':
        if args.epochs % 10 == 0:
            for i, train_loss_acc in enumerate(zip(train_losses, train_accuracies)):
                print(f'{gcn.map_id2labels[i]}_train_loss', round(train_loss_acc[0].detach().item(), 4))
                print(f'{gcn.map_id2labels[i]}_train_accuracy', round(train_loss_acc[1].item(), 2) * 100)
            for i, val_loss_acc in enumerate(zip(val_losses, val_accuracies)):
                print(f'{gcn.map_id2labels[i]}_val_loss', round(val_loss_acc[0].detach().item(), 4))
                print(f'{gcn.map_id2labels[i]}_val_accuracy', round(val_loss_acc[1].item(), 2) * 100)
            for i, test_loss_acc in enumerate(zip(test_losses, test_accuracies)):
                print(f'{gcn.map_id2labels[i]}_test_loss', round(test_loss_acc[0].detach().item(), 4))
                print(f'{gcn.map_id2labels[i]}_test_accuracy', round(test_loss_acc[1].item(), 2) * 100)
    if args.mode == 'single_task':
        if args.epochs % 10 == 0:
            print(f'{args.label}_train_loss', round(train_losses[0].detach().item(), 4))
            print(f'{args.label}_train_accuracy', round(train_accuracies[0].item(), 2) * 100)
            print(f'{args.label}_val_loss', round(val_losses[0].detach().item(), 4))
            print(f'{args.label}_val_accuracy', round(val_accuracies[0].item(), 2) * 100)
            print(f'{args.label}_test_loss', round(test_losses[0].detach().item(), 4))
            print(f'{args.label}_test_accuracy', round(test_accuracies[0].item(), 2) * 100)

  0%|          | 1/200 [00:03<11:27,  3.45s/it]

genre_train_loss 4.7222
genre_train_accuracy 1.0
genre_val_loss 4.6047
genre_val_accuracy 1.0
genre_test_loss 4.6205
genre_test_accuracy 1.0


  1%|          | 2/200 [00:06<11:18,  3.42s/it]

genre_train_loss 4.456
genre_train_accuracy 1.0
genre_val_loss 4.3677
genre_val_accuracy 1.0
genre_test_loss 4.3985
genre_test_accuracy 1.0


  2%|▏         | 3/200 [00:10<12:14,  3.73s/it]

...

genre_train_loss 0.2373
genre_train_accuracy 97.0
genre_val_loss 0.2448
genre_val_accuracy 98.0
genre_test_loss 1.8324
genre_test_accuracy 66.0


 46%|████▌     | 91/200 [05:07<06:59,  3.85s/it]

genre_train_loss 0.2323
genre_train_accuracy 97.0
genre_val_loss 0.2399
genre_val_accuracy 98.0
genre_test_loss 1.8377
genre_test_accuracy 66.0


 46%|████▌     | 92/200 [05:11<07:15,  4.03s/it]

genre_train_loss 0.2274
genre_train_accuracy 97.0
genre_val_loss 0.2351
genre_val_accuracy 98.0
genre_test_loss 1.8432
genre_test_accuracy 66.0
